# PRM of Let's Verify Step by Step

In [1]:
!python -c "import torch; print(torch.cuda.is_available())"

True


__Origional PRM Implementation__

1. Pretrain a model on the MATH like dataset for both actor and critic as MathMix, so it can output step by step like solutions (I dont have the access to MathMix Dataset, maybe I can use [GSM8K](https://github.com/openai/grade-school-math) as a substitute)
2. Fine-tune the critic model on the classification task of the PRM800K, so it output labels as positive, neutral, negative
3. Use the critic model to implement best of N sampling from the actor model, so it can generate the best step by step solutions

Actor Model uses solution from few shot generated data to tune the base model. So it learns the format. Basically they did not teach any new skills to Actor

__Proposed PRM Implementation__
1. Pretrain TinyLlama on GSM8K dataset
2. Fine-tune the critic model on the classification task of the PRM800K, so it output labels as positive, neutral, negative
3. Use the critic model to implement best of N sampling from the actor model, so it can generate the best step by step solutions

In [1]:
import torch; torch.version.cuda


'11.8'

Maybe I can also use the temporal structure for this in the Q function. But I need to change the PRM from product score to sum score.

## Fine-tuning the base model
Install the unsloth first to help with the memory
Based on the paper, they used GPT-4 without RLHF. So it is pretrained solely to predict the next token.
[TinyLlama 1.1B intermediate step 1431k 3T](https://huggingface.co/unsloth/tinyllama) should be perfect for this task

In [1]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.

# 4bit pre quantized models we support for 4x faster downloading + no OOMs.
# fourbit_models = [
#     "unsloth/mistral-7b-bnb-4bit",
#     "unsloth/mistral-7b-instruct-v0.2-bnb-4bit",
#     "unsloth/llama-2-7b-bnb-4bit",
#     "unsloth/llama-2-13b-bnb-4bit",
#     "unsloth/codellama-34b-bnb-4bit",
#     "unsloth/tinyllama-bnb-4bit",
#     "unsloth/gemma-7b-bnb-4bit", # New Google 6 trillion tokens model 2.5x faster!
#     "unsloth/gemma-2b-bnb-4bit",
# ] # More models at https://huggingface.co/unsloth

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-Instruct", # "unsloth/tinyllama" for 16bit loading
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
    # token = "hf_...", # use one if using gated models like meta-llama/Llama-2-7b-hf
)

#well for llama3 'pre-trained models usually do not stop completions naturally.'
#https://github.com/meta-llama/llama3/blob/main/example_text_completion.py

/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.642 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.35it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
#Use LoRA to reduce memory usage:
model = FastLanguageModel.get_peft_model(
    model,
    r = 32, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = ["q_proj", "k_proj", "v_proj", "o_proj",
                      "gate_proj", "up_proj", "down_proj",],
    lora_alpha = 32,
    lora_dropout = 0, # Currently only supports dropout = 0
    bias = "none",    # Currently only supports bias = "none"
    use_gradient_checkpointing = "unsloth", # @@@ IF YOU GET OUT OF MEMORY - set to True @@@
    random_state = 3407,
    use_rslora = False,  # We support rank stabilized LoRA
    loftq_config = None, # And LoftQ
)

Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.


Train on the completion only for the GSM8K dataset

Use the completion training from [Huggingface](https://huggingface.co/docs/trl/sft_trainer#train-on-completions-only)

For unsloth, it is a bit unsure about whether we can use [DataCollatorForCompletionOnlyLM with packing = True](https://github.com/unslothai/unsloth/issues/109), so I am going to follow the colab example from unsloth first

In [3]:
# prompt = """
# ### Input:
# {}

# ### Response:
# {}"""

# BOS_TOKEN = tokenizer.bos_token
# prompt = BOS_TOKEN + prompt
# print(prompt)

prompt = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}<|eot_id|>

"""
prompt.format("test question", "answers")


'<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>\n\ntest question<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nanswers<|eot_id|>\n\n'

In [9]:
# FastLanguageModel.for_inference(model) # Enable native 2x faster inference
# inputs = tokenizer(
# [
#     prompt.format(
#         "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?", # Question
#         "", # output - leave this blank for generation!
#     )
# ], return_tensors = "pt").to("cuda")

In [11]:
# from transformers import TextStreamer
# text_streamer = TextStreamer(tokenizer)
# # _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>"))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>

A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

<|eot_id|>

Let's break it down step by step:

1. The robe takes 2 bolts of blue fiber.
2. It takes half that much white fiber. To find half of 2, we can divide 2 by 2:

2 ÷ 2 = 1

So, the robe takes 1 bolt of white fiber.

3. To find the total number of bolts, we add the blue and white fibers:

2 (blue) + 1 (white) = 3

Therefore, the robe takes a total of 3 bolts of fiber.<|eot_id|>


In [4]:
from trl import SFTTrainer, DataCollatorForCompletionOnlyLM


# EOS_TOKEN = tokenizer.eos_token
def formatting_prompts_func(example):
    questions = example['question']
    answers = example['answer']
    output_texts = []
    for questions, answers in zip(questions, answers):
        output_text = prompt.format(questions, answers)
        output_texts.append(output_text)
    return { "text" : output_texts, }
pass
# response_template = " ### Answer:"
# collator = DataCollatorForCompletionOnlyLM(response_template, tokenizer=tokenizer)
from datasets import load_dataset
dataset = load_dataset("gsm8k", 'main', split='train')
dataset = dataset.map(formatting_prompts_func, batched = True,)

Map: 100%|██████████| 7473/7473 [00:00<00:00, 372624.25 examples/s]


In [5]:
#check dataset
dataset.__len__()

7473

Training

Epoch consideration:
From the origional paper: "Note that when training smaller models, as in Section 4, we use a slightly smaller variant of MathMix that excludes the critiques data and only consists of 1B tokens. For our large models experiments, we train on MathMix for roughly 3B tokens (2 epochs). For our small models experiments, we train for 6 epochs (roughly 6.6B tokens)."

The model they used is similar to GPT-4, but "The small-scale base models are similar in design to GPT-4, but they were pretrained with roughly 200 times less compute.". So my guess is about 8.8B model parameters.

If we are using 1.1B model parameters we would need like 1.1/8.8 * 6.6 = 0.825B tokens. GSM8K, let's say each problem is about 85 tokens. So we have 8K problems that's 680K tokens. 0.825B/0.68M = 1213 epochs. Maybe it will over fit.

My guess is the loss around 1 would be good. I tried 7 epochs in totoal and the response has the desired format. I am gonna try 10 epochs to see if it can get better.

In [6]:
from trl import SFTTrainer
from transformers import TrainingArguments
from transformers.utils import logging
logging.set_verbosity_info()

trainer = SFTTrainer(
    model = model,
    tokenizer = tokenizer,
    train_dataset = dataset,
    dataset_text_field = "text",
    max_seq_length = max_seq_length,
    dataset_num_proc = 2,
    packing = False, 
    args = TrainingArguments(
        per_device_train_batch_size = 2,
        gradient_accumulation_steps = 4,
        warmup_ratio = 0.1,
        num_train_epochs = 1,
        learning_rate = 2e-5,
        fp16 = not torch.cuda.is_bf16_supported(),
        bf16 = torch.cuda.is_bf16_supported(),
        logging_steps = 100,
        save_steps= 2000,
        optim = "adamw_8bit",
        weight_decay = 0.1,
        lr_scheduler_type = "linear",
        seed = 3407,
        output_dir = "checkpoints/llama3-8b-gsm8k",
    ),
)

PyTorch: setting up devices
The default value for the training argument `--report_to` will change in v5 (from all installed integrations to none). In v5, you will need to use `--report_to all` to get the same behavior as now. You should start updating your code and make this info disappear :-).
Map (num_proc=2): 100%|██████████| 7473/7473 [00:00<00:00, 7679.44 examples/s]
Using auto half precision backend


In [7]:
#@title Show current memory stats
gpu_stats = torch.cuda.get_device_properties(0)
start_gpu_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
max_memory = round(gpu_stats.total_memory / 1024 / 1024 / 1024, 3)
print(f"GPU = {gpu_stats.name}. Max memory = {max_memory} GB.")
print(f"{start_gpu_memory} GB of memory reserved.")

GPU = NVIDIA GeForce RTX 4090. Max memory = 23.642 GB.
15.404 GB of memory reserved.


In [8]:
trainer_stats = trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs = 1
   \\   /|    Num examples = 7,473 | Num Epochs = 1
O^O/ \_/ \    Batch size per device = 2 | Gradient Accumulation steps = 4
\        /    Total batch size = 8 | Total steps = 934
 "-____-"     Number of trainable parameters = 83,886,080


Automatic Weights & Biases logging enabled, to disable set os.environ["WANDB_DISABLED"] = "true"
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jq394 (neurorunner). Use `wandb login --relogin` to force relogin


Step,Training Loss
100,1.687700
200,0.910600
300,0.834100
400,0.837000
500,0.820000
600,0.818500
700,0.824700
800,0.821500
900,0.815200




Training completed. Do not forget to share your model on huggingface.co/models =)




In [9]:
#@title Show final memory and time stats
used_memory = round(torch.cuda.max_memory_reserved() / 1024 / 1024 / 1024, 3)
used_memory_for_lora = round(used_memory - start_gpu_memory, 3)
used_percentage = round(used_memory         /max_memory*100, 3)
lora_percentage = round(used_memory_for_lora/max_memory*100, 3)
print(f"{trainer_stats.metrics['train_runtime']} seconds used for training.")
print(f"{round(trainer_stats.metrics['train_runtime']/60, 2)} minutes used for training.")
print(f"Peak reserved memory = {used_memory} GB.")
print(f"Peak reserved memory for training = {used_memory_for_lora} GB.")
print(f"Peak reserved memory % of max memory = {used_percentage} %.")
print(f"Peak reserved memory for training % of max memory = {lora_percentage} %.")

766.9912 seconds used for training.
12.78 minutes used for training.
Peak reserved memory = 16.689 GB.
Peak reserved memory for training = 1.285 GB.
Peak reserved memory % of max memory = 70.59 %.
Peak reserved memory for training % of max memory = 5.435 %.


Inference

In [24]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
inputs = tokenizer(
[
    prompt.format(
        "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?", # Question
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = model.generate(**inputs, max_new_tokens = 128, use_cache = True)
tokenizer.batch_decode(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>\n\nA robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?<|eot_id|><|start_header_id|>assistant<|end_header_id|>\n\nIt takes 2 bolts of blue fiber and 2/2 = <<2/2=1>>1 bolt of white fiber.\nSo it takes 2+1 = <<2+1=3>>3 bolts of fiber in total.\n#### 3<|eot_id|>\n\nA robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?\nIt takes 2 bolts of blue fiber and 2/2 = <<2/2=1>>1 bolt of white fiber.\nSo it takes 2+1 = <<2+1=3>>3 bolts of fiber in total.\n']


In [29]:
# alpaca_prompt = Copied from above
FastLanguageModel.for_inference(model) # Enable native 2x faster inference
prompt_inference = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""

inputs = tokenizer(
[
    prompt_inference.format(
        "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?", # Question
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
# _ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, do_sample=True, temperature=0.5, top_k=40, eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>"))

Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>

A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?<|eot_id|><|start_header_id|>assistant<|end_header_id|>



The robe takes 2 x 1/2 = <<2*1/2=1>>1 bolt of white fiber.
So in total, it takes 1 + 2 = <<1+2=3>>3 bolts of fiber.
#### 3<|eot_id|>


In [27]:
# model.save_pretrained("checkpoints/llama3-8b-gsm8k-1epoch-lora") # Local saving


loading configuration file config.json from cache at /home/jqi/.cache/huggingface/hub/models--unsloth--llama-3-8b-Instruct/snapshots/c0e1bc512b26491535f48c9cee041e1c4db9d81d/config.json
Model config LlamaConfig {
  "_name_or_path": "meta-llama/Meta-Llama-3-8B-Instruct",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 128000,
  "eos_token_id": 128001,
  "hidden_act": "silu",
  "hidden_size": 4096,
  "initializer_range": 0.02,
  "intermediate_size": 14336,
  "max_position_embeddings": 8192,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 32,
  "num_key_value_heads": 8,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 500000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.39.3",
  "use_cache": true,
  "vocab_size": 128256
}



In [137]:
if True: model.save_pretrained_merged("checkpoints/llama3-8b-gsm8k-1epoch", tokenizer, save_method = "merged_16bit",)


Unsloth: Merging 4bit and LoRA weights to 16bit...
Unsloth: Will use up to 39.84 out of 62.51 RAM for saving.


100%|██████████| 22/22 [00:00<00:00, 357.00it/s]
tokenizer config file saved in tinyLlama-GSM8K-5epochs/tokenizer_config.json
Special tokens file saved in tinyLlama-GSM8K-5epochs/special_tokens_map.json
Model config LlamaConfig {
  "_name_or_path": "unsloth/tinyllama",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.39.3",
  "unsloth_version": "2024.4",
  "use_cache": true,
  "vocab_size": 32000
}

Configuration saved in tinyLlama-GSM8K-5epo

Unsloth: Saving tokenizer... Done.
Unsloth: Saving model... This might take 5 minutes for Llama-7b...


Model weights saved in tinyLlama-GSM8K-5epochs/model.safetensors


Done.


Load the model back

In [1]:
#Now if you want to load the LoRA adapters we just saved for inference, set False to True:
from unsloth import FastLanguageModel
import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.
if True:
    from unsloth import FastLanguageModel
    model, tokenizer = FastLanguageModel.from_pretrained(
        model_name = "checkpoints/llama3-8b-gsm8k-1epoch-lora", # YOUR MODEL YOU USED FOR TRAINING
        max_seq_length = max_seq_length,
        dtype = dtype,
        load_in_4bit = load_in_4bit,
    )
    FastLanguageModel.for_inference(model) # Enable native 2x faster inference

# alpaca_prompt = You MUST copy from above!
prompt_inference = """<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>

{}<|eot_id|><|start_header_id|>assistant<|end_header_id|>

{}"""

inputs = tokenizer(
[
    prompt_inference.format(
        "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?", # Question
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

from transformers import TextStreamer
text_streamer = TextStreamer(tokenizer)
_ = model.generate(**inputs, streamer = text_streamer, max_new_tokens = 128, do_sample=True, temperature=0.5, top_k=40, eos_token_id = tokenizer.convert_tokens_to_ids("<|eot_id|>"))

/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.642 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  2.48it/s]
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
Unsloth 2024.4 patched 32 layers with 32 QKV layers, 32 O layers and 32 MLP layers.
Setting `pad_token_id` to `eos_token_id`:128009 for open-end generation.


<|begin_of_text|><|start_header_id|>system<|end_header_id|>

You are a helpful assistant to solve math problems step by step <|eot_id|><|start_header_id|>user<|end_header_id|>

A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?<|eot_id|><|start_header_id|>assistant<|end_header_id|>

It takes 2/2 = <<2/2=1>>1 bolt of white fiber.
So it takes 2+1 = <<2+1=3>>3 bolts of fiber in total.
#### 3<|eot_id|>


In [2]:
outputs = model.generate(**inputs, max_new_tokens = 64, use_cache = True)
print(outputs)

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


tensor([[  198, 14711,  5688,   512,    32, 63719,  5097,   220,    17, 49939,
           315,  6437, 24722,   323,  4376,   430,  1790,  4251, 24722,    13,
          2650,  1690, 49939,   304,  2860,  1587,   433,  1935,  1980, 14711,
          6075,   512,  2181,  5097,   220,    17,     9,    15,    13,    20,
            28,  2501,    17,     9,    15,    13,    20,    28,    16,  2511,
            16, 32942,   315,  4251, 24722,   627,  4516,   433,  5097,   220,
            17,    10,    16,    28,  2501,    17,    10,    16,    28,    18,
          2511,    18, 49939,   304,  2860,   627,   827,   220,    18, 49939,
           198, 68431,   220,    18, 49939,   198,   827,   220,    18, 49939,
           198, 68431,   220,    18, 49939,   198]], device='cuda:0')


In [145]:
from unsloth import FastLanguageModel
model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "tinyLlama-GSM8K-10epochs", # YOUR MODEL YOU USED FOR TRAINING
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)
FastLanguageModel.for_inference(model) # Enable native 2x faster inference

loading configuration file tinyLlama-GSM8K-10epochs/config.json
Model config LlamaConfig {
  "_name_or_path": "tinyLlama-GSM8K-10epochs",
  "architectures": [
    "LlamaForCausalLM"
  ],
  "attention_bias": false,
  "attention_dropout": 0.0,
  "bos_token_id": 1,
  "eos_token_id": 2,
  "hidden_act": "silu",
  "hidden_size": 2048,
  "initializer_range": 0.02,
  "intermediate_size": 5632,
  "max_position_embeddings": 2048,
  "model_type": "llama",
  "num_attention_heads": 32,
  "num_hidden_layers": 22,
  "num_key_value_heads": 4,
  "pad_token_id": 0,
  "pretraining_tp": 1,
  "rms_norm_eps": 1e-05,
  "rope_scaling": null,
  "rope_theta": 10000.0,
  "tie_word_embeddings": false,
  "torch_dtype": "bfloat16",
  "transformers_version": "4.39.3",
  "unsloth_version": "2024.4",
  "use_cache": true,
  "vocab_size": 32000
}

loading configuration file tinyLlama-GSM8K-10epochs/config.json
Model config LlamaConfig {
  "_name_or_path": "tinyLlama-GSM8K-10epochs",
  "architectures": [
    "LlamaForCau

==((====))==  Unsloth: Fast Llama patching release 2024.4
   \\   /|    GPU: NVIDIA GeForce RTX 4090. Max memory: 23.642 GB. Platform = Linux.
O^O/ \_/ \    Pytorch: 2.2.2. CUDA = 8.9. CUDA Toolkit = 11.8.
\        /    Bfloat16 = TRUE. Xformers = 0.0.25.post1. FA = True.
 "-____-"     Free Apache license: http://github.com/unslothai/unsloth


All model checkpoint weights were used when initializing LlamaForCausalLM.

All the weights of LlamaForCausalLM were initialized from the model checkpoint at tinyLlama-GSM8K-10epochs.
If your task is similar to the task the model of the checkpoint was trained on, you can already use LlamaForCausalLM for predictions without further training.
loading configuration file tinyLlama-GSM8K-10epochs/generation_config.json
Generate config GenerationConfig {
  "bos_token_id": 1,
  "eos_token_id": 2,
  "max_length": 2048,
  "pad_token_id": 0
}

loading file tokenizer.model
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
loading file tokenizer.json
loading file tokenizer.model
loading file tokenizer.json
loading file added_tokens.json
loading file special_tokens_map.json
loading file tokenizer_config.json
tokenizer config file saved in _unsloth_sentencepiece_temp/tinyLlama-GSM8K-10epochs/tokenizer_config.json
Special tokens file saved in _unsl

In [54]:
print(model)

LlamaForCausalLM(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRMSNorm()
  )
  (lm_head): Line

# Fine Tuning the Critic Model on PRM800K

1. Training the PRM
- Token-Based Prediction: After each step in the solution, the PRM predicts a token representing the correctness of the step. This is typically a classification task where the output token could represent categories such as correct, incorrect, or some probabilistic label.
- Training Objective: The model is trained to maximize the log-likelihood of these correctness tokens. This means that during training, you'll use a loss function appropriate for classification (such as cross-entropy) where the targets are the correctness tokens.
- Model Architecture: The PRM can be any standard language model that supports token prediction, such as those based on the Transformer architecture. You train this model using a standard language model training pipeline.

we deliberately choose to supervise only up to the first incorrect step.

So basically they are training a model with a classifier head. The head is giving the score of the reward
- Model predicts a single next token with the input "We train our PRMs by fine-tuning the MathMix model to predict the probability of positive, negative, and neutral labels given a solution prefix ending in one of our labeled steps."
- Add a classifier head to the model to predict the score of the next token
- Train the model with the cross-entropy loss
- we found low learning rates were important to stable PRM training.

so SFTTrainer probably wont work. I should use the Trainer from huggingface
If I can use unsloth lora then I can make much faster training

What I need to achieve for the model are:
- Initialize a classification head ✅
- Modifiy the dataset to adjust for the training
- Use LoRA and Unsloth to speed up the training and reduce memory
- Modify the training pipeline
- Train the model with the cross-entropy loss on three labels
- plot the loss and see if it is converging
- Save the model

Based on the dataset results I probably can use Birchlabs/openai-prm800k-stepwise-critic dataset since it is already in the format I need

There are a lot of null values in rating so I need to drop those

Data Preprocessing

## LoRA Training

In [1]:
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM
import torch
from transformers import BitsAndBytesConfig
config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
    bnb_4bit_compute_dtype=torch.bfloat16,
)

model = LlamaForSequenceClassification.from_pretrained("unsloth/llama-3-8b", num_labels = 3, quantization_config=config)
# model = LlamaForCausalLM.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")

tokenizer = AutoTokenizer.from_pretrained("unsloth/llama-3-8b", padding_side='right', use_fast          = False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda
# model.to(device)

from peft import prepare_model_for_kbit_training

model = prepare_model_for_kbit_training(model)

/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Loading checkpoint shards: 100%|██████████| 4/4 [00:01<00:00,  3.09it/s]
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at unsloth/llama-3-8b and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [2]:
#LoRA config
from peft import (
        get_peft_model, 
        prepare_model_for_kbit_training, 
        LoraConfig,
        TaskType
    )

config = LoraConfig(
r= 32,
lora_alpha          = 32,
target_modules      = ["q_proj", "k_proj", "v_proj", "o_proj", "gate_proj", "up_proj", "down_proj"],
bias                = 'none',
task_type           = TaskType.SEQ_CLS,
modules_to_save     = ["score"],
)

model = get_peft_model(model, config)

In [3]:
model.print_trainable_parameters()


trainable params: 83,898,368 || all params: 7,588,835,328 || trainable%: 1.1055499872351426


In [5]:
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM
import torch
import wandb
import os

wandb.login()
os.environ['WANDB_PROJECT'] = 'LLMRL'

EOS_TOKEN = tokenizer.eos_token  # End-of-Sequence token
prompt = """
### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    texts = []
    labels = []
    
    for instruction, responses, next_response, rating in zip(examples['instruction'], examples['responses'], examples['next_response'], examples['rating']):
        # Combine all responses and the next response into a single string with newline separation
        combined_responses = "\n".join(responses) + "\n" + next_response
        
        # Format the text with the prompt template
        text = prompt.format(instruction, combined_responses) + EOS_TOKEN
        texts.append(text)
        labels.append(rating + 1)  # Convert ratings to labels by adding 1

    # Tokenize all texts at once using the tokenizer
    model_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=512)

    # Add labels to the model inputs
    model_inputs['labels'] = labels
    
    return model_inputs

from datasets import load_dataset

# Load and preprocess the dataset
dataset = load_dataset("Birchlabs/openai-prm800k-stepwise-critic", split='test')
dataset = dataset.filter(lambda x: x['rating'] is not None)  # Filter entries without ratings
dataset = dataset.map(formatting_prompts_func, batched=True)  # Apply the preprocessing function

from transformers import TrainerCallback

class PrintLossCallback(TrainerCallback):
    """Print the training loss at each logging step."""
    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            print(f"Step {state.global_step}: Loss {logs['loss']:.4f}")
            


Map: 100%|██████████| 31338/31338 [00:03<00:00, 8262.09 examples/s]


TinyLlma GPU VRAM requirements for finetuning:
| context length | Without Lora (GB) | Lora (GB) |
|----------------|-------------------|-----------|
| 512            | 12.3              | 6.6       |
| 1024           | 13.4              | 10        |
| 2048           | 18.7              | 16.9      |
| 4096           | 31.7              | 30.5      |
| 8192           | 54.2              | 57.8      |


In [6]:
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import tqdm


# Define the compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=labels)



# Define TrainingArguments
training_args = TrainingArguments(
    output_dir='./checkpoints/tinyLlama-critic-lora',          # Output directory
    num_train_epochs=1,              # Total number of training epochs
    per_device_train_batch_size=4,   # Batch size per device during training
    gradient_accumulation_steps=4,       # Number of update steps to accumulate before performing a backward/update pass
    warmup_ratio=0.03,                # Number of warmup steps for learning rate scheduler
    learning_rate=4e-6,                   # Learning rate for the optimizer
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_strategy='steps',
    logging_steps=10,
    evaluation_strategy="no",     # Evaluate each `logging_steps`
    save_strategy="steps",           # Save a checkpoint at the end of each epoch
    save_steps=10000,                   # Number of updates steps before saving
    save_total_limit=1,              # Maximum number of checkpoints to keep
    # load_best_model_at_end=True,     # Load the best model at the end of training based on metric (if evaluation is enabled)
    report_to="wandb",               # Enable logging to W&B
)

# Initialize Trainer
trainer = Trainer(
    model=model,                    # The instantiated 🤗 Transformers model to be trained
    args=training_args,             # Training arguments, defined above
    train_dataset=dataset,          # Training dataset
    # eval_dataset=dataset,           # Evaluation dataset
    # compute_metrics=compute_metrics, # The callback that computes metrics of interest
    # callbacks=[PrintLossCallback()]
)



/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


OutOfMemoryError: CUDA out of memory. Tried to allocate 224.00 MiB. GPU 0 has a total capacity of 23.64 GiB of which 34.00 MiB is free. Including non-PyTorch memory, this process has 23.29 GiB memory in use. Of the allocated memory 22.89 GiB is allocated by PyTorch, and 25.96 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [ ]:
# Train the model
trainer.train()

Step,Training Loss
10,1.182300
20,1.081300


KeyboardInterrupt: 

In [10]:

# Assuming the Trainer has completed training
model_path = "./checkpoints/tinyLlama-critic-lora"
model.save_pretrained(model_path)
tokenizer.save_pretrained(model_path)

('./checkpoints/tinyLlama-critic-lora/tokenizer_config.json',
 './checkpoints/tinyLlama-critic-lora/special_tokens_map.json',
 './checkpoints/tinyLlama-critic-lora/tokenizer.model',
 './checkpoints/tinyLlama-critic-lora/added_tokens.json')

In [13]:
from peft import AutoPeftModelForSequenceClassification

model = AutoPeftModelForSequenceClassification.from_pretrained('/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic-lora', num_labels = 3)

Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at checkpoints/tinyLlama-GSM8K-10epochs and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [15]:
model.score.weight

Parameter containing:
tensor([[ 0.0246, -0.0025,  0.0143,  ...,  0.0093, -0.0189,  0.0170],
        [ 0.0079, -0.0123,  0.0532,  ...,  0.0119,  0.0128, -0.0012],
        [-0.0122,  0.0111,  0.0271,  ...,  0.0155, -0.0049, -0.0119]],
       requires_grad=True)

## Actual Training

In [1]:
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM
import torch
model = LlamaForSequenceClassification.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", num_labels = 3)
# model = LlamaForCausalLM.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")

tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", padding_side='right', use_fast          = False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda
model.to(device)


/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of LlamaForSequenceClassification were not initialized from the model checkpoint at checkpoints/tinyLlama-GSM8K-10epochs and are newly initialized: ['score.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
You are using the default legacy behaviour of the <class 'transformers.models.llama.tokenization_llama.LlamaTokenizer'>. This is expected, and simply means that the `legacy` (previous) behavior will be used so nothing changes for you. If you want to use the new behaviour, set `legacy=False`. This should only be set if you understand what it means, and thoroughly read the reason why this was added as explained in https://github.com/huggingface

LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [2]:
from transformers import AutoTokenizer


EOS_TOKEN = tokenizer.eos_token  # End-of-Sequence token
prompt = """
### Input:
{}

### Response:
{}"""

def formatting_prompts_func(examples):
    texts = []
    labels = []
    
    for instruction, responses, next_response, rating in zip(examples['instruction'], examples['responses'], examples['next_response'], examples['rating']):
        # Combine all responses and the next response into a single string with newline separation
        combined_responses = "\n".join(responses) + "\n" + next_response
        
        # Format the text with the prompt template
        text = prompt.format(instruction, combined_responses) + EOS_TOKEN
        texts.append(text)
        labels.append(rating + 1)  # Convert ratings to labels by adding 1

    # Tokenize all texts at once using the tokenizer
    model_inputs = tokenizer(texts, padding="max_length", truncation=True, max_length=512)

    # Add labels to the model inputs
    model_inputs['labels'] = labels
    
    return model_inputs


In [3]:
from datasets import load_dataset

# Load and preprocess the dataset
dataset = load_dataset("Birchlabs/openai-prm800k-stepwise-critic", split='train')
dataset = dataset.filter(lambda x: x['rating'] is not None)  # Filter entries without ratings
dataset = dataset.map(formatting_prompts_func, batched=True)  # Apply the preprocessing function


In [4]:
from transformers import TrainerCallback

class PrintLossCallback(TrainerCallback):
    """Print the training loss at each logging step."""
    def on_log(self, args, state, control, logs=None, **kwargs):
        if 'loss' in logs:
            print(f"Step {state.global_step}: Loss {logs['loss']:.4f}")


In [8]:
from transformers import TrainingArguments, Trainer
from datasets import load_dataset
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate
import tqdm
import wandb
wandb.login()
%env WANDB_PROJECT=LLMRL

# Define the compute_metrics function
def compute_metrics(eval_pred):
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)
    accuracy = evaluate.load("accuracy")
    return accuracy.compute(predictions=predictions, references=labels)



# Define TrainingArguments
training_args = TrainingArguments(
    output_dir='./checkpoints',          # Output directory
    num_train_epochs=1,              # Total number of training epochs
    per_device_train_batch_size=2,   # Batch size per device during training
    warmup_ratio=0.03,                # Number of warmup steps for learning rate scheduler
    learning_rate=1e-6,                   # Learning rate for the optimizer
    weight_decay=0.01,               # Strength of weight decay
    logging_dir='./logs',            # Directory for storing logs
    logging_strategy='steps',
    logging_steps=10,
    evaluation_strategy="epoch",     # Evaluate each `logging_steps`
    save_strategy="epoch",           # Save a checkpoint at the end of each epoch
    save_total_limit=1,              # Maximum number of checkpoints to keep
    load_best_model_at_end=True,     # Load the best model at the end of training based on metric (if evaluation is enabled)
    report_to="wandb",               # Enable logging to W&B
)

# Initialize Trainer
trainer = Trainer(
    model=model,                    # The instantiated 🤗 Transformers model to be trained
    args=training_args,             # Training arguments, defined above
    train_dataset=dataset,          # Training dataset
    eval_dataset=dataset,           # Evaluation dataset
    compute_metrics=compute_metrics, # The callback that computes metrics of interest
    callbacks=[PrintLossCallback()]
)

# Train the model
trainer.train()


env: WANDB_PROJECT=LLMRL


/home/jqi/anaconda3/envs/llmrl/lib/python3.10/site-packages/accelerate/accelerator.py:436: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


Epoch,Training Loss,Validation Loss


Step 10: Loss 1.6106
Step 20: Loss 1.2224
Step 30: Loss 1.2879
Step 40: Loss 1.3943
Step 50: Loss 1.0252
Step 60: Loss 1.0790
Step 70: Loss 0.7601
Step 80: Loss 1.3390
Step 90: Loss 1.3616
Step 100: Loss 0.8222
Step 110: Loss 0.9185
Step 120: Loss 0.9664
Step 130: Loss 1.1814
Step 140: Loss 2.0675
Step 150: Loss 1.3662
Step 160: Loss 1.1383
Step 170: Loss 0.6260
Step 180: Loss 0.9413
Step 190: Loss 0.6430
Step 200: Loss 2.6985
Step 210: Loss 2.8711
Step 220: Loss 2.0896
Step 230: Loss 1.9361
Step 240: Loss 1.1050
Step 250: Loss 1.1790
Step 260: Loss 1.0712
Step 270: Loss 1.8108
Step 280: Loss 1.1018
Step 290: Loss 0.9825
Step 300: Loss 1.3303
Step 310: Loss 1.7695
Step 320: Loss 1.5156
Step 330: Loss 0.6667
Step 340: Loss 1.2210
Step 350: Loss 1.2253
Step 360: Loss 1.9313
Step 370: Loss 1.1900
Step 380: Loss 1.0701
Step 390: Loss 0.9601
Step 400: Loss 1.2723
Step 410: Loss 0.9714
Step 420: Loss 1.2677
Step 430: Loss 1.2334
Step 440: Loss 1.5288
Step 450: Loss 1.2223
Step 460: Loss 1.32

KeyboardInterrupt: 

# PRM Evaluation

Based on the paper, the evaluation is using N sampling method. So I need to implement the N sampling method to get the best solution
Generator: Generate the entire solution for N times
Critic: Evaluate the solution for N times. 

We can test it with the grading method from the dataset. Also we can directly select solution from the dataset "is_solution" column
The output of the model is in the format of 


'### [Solution]'

So technically I can just extract the end of the output and use the grader to verify the solution

In [7]:
import sys
sys.path.append('Data/prm800k/prm800k')
from grading import grader


In [26]:
model_answer =  " 320,000"

gt_answer = "320,000"

In [27]:
grader.grade_answer(model_answer, gt_answer)

True

## Generator
Generator generate the solution for N times. The critic evaluate the solution for N times. The critic will output the score of the solution. The generator will output the solution with the highest score

In [27]:
# from unsloth import FastLanguageModel
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM

import torch
max_seq_length = 2048 # Choose any! We auto support RoPE Scaling internally!
dtype = None # None for auto detection. Float16 for Tesla T4, V100, Bfloat16 for Ampere+
load_in_4bit = False # Use 4bit quantization to reduce memory usage. Can be False.


generator = LlamaForCausalLM.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs")
tokenizer = AutoTokenizer.from_pretrained("checkpoints/tinyLlama-GSM8K-10epochs", padding_side='right', use_fast = False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda

generator.to(device)

#set generator to inference mode
generator.eval()

prompt = """
### Input:
{}

### Response:
{}"""

inputs = tokenizer(
[
    prompt.format(
        "A robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?", # Question
        "", # output - leave this blank for generation!
    )
], return_tensors = "pt").to("cuda")

outputs = generator.generate(**inputs, max_new_tokens = 256, use_cache = True, do_sample=True, temperature=0.5, top_k=40)

In [28]:
print(tokenizer.batch_decode(outputs))

['<s> \n### Input:\nA robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?\n\n### Response:\nFirst find how many bolts of blue fiber the robe takes: 2 bolts of blue fiber * 2/2 = <<2*2=4>>4 bolts\nThen find how many bolts of white fiber the robe takes: 4 bolts of white fiber / 2 = <<4/2=2>>2 bolts\nThen add the two bolt counts to find the total number of bolts: 2 bolts of blue fiber + 2 bolts of white fiber = <<2+2=4>>4 bolts\n#### 4</s>']


In [29]:
ouput_text = tokenizer.batch_decode(outputs)

In [30]:
for i in range(len(ouput_text)):
    ouput_text[i] = ouput_text[i][4:-4]


## Critic
First I need to test whether critic actually work for the wrong solutions

In [15]:
from transformers import LlamaForSequenceClassification, AutoTokenizer, LlamaForCausalLM
import torch
critic = LlamaForSequenceClassification.from_pretrained("/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic")
critic.eval()

critic_tokenizer = AutoTokenizer.from_pretrained("/home/jqi/Github/LLMRL/checkpoints/tinyLlama-critic", padding_side='right', use_fast= False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
#to cuda
critic.to(device)


LlamaForSequenceClassification(
  (model): LlamaModel(
    (embed_tokens): Embedding(32000, 2048, padding_idx=0)
    (layers): ModuleList(
      (0-21): 22 x LlamaDecoderLayer(
        (self_attn): LlamaSdpaAttention(
          (q_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (k_proj): Linear(in_features=2048, out_features=256, bias=False)
          (v_proj): Linear(in_features=2048, out_features=256, bias=False)
          (o_proj): Linear(in_features=2048, out_features=2048, bias=False)
          (rotary_emb): LlamaRotaryEmbedding()
        )
        (mlp): LlamaMLP(
          (gate_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (up_proj): Linear(in_features=2048, out_features=5632, bias=False)
          (down_proj): Linear(in_features=5632, out_features=2048, bias=False)
          (act_fn): SiLU()
        )
        (input_layernorm): LlamaRMSNorm()
        (post_attention_layernorm): LlamaRMSNorm()
      )
    )
    (norm): LlamaRM

In [7]:
prompt = """
### Input:
{}

### Response:
{}"""

Question = "How many positive two-digit integers leave a remainder of 2 when divided by 8?"
Answer = [ "So if a number leaves a remainder of 2 when divided by 8, it's of the form 8n+2.", "So we want to know the number of positive two-digit integers of the form 8n+2.", "I think we should just plug in numbers and see what happens.", "Ok let's start with n=1.", "8*1+2=10 which is a two-digit integer.", "Let's try n=2.", "8*2+2=18 which is also a two-digit integer." ]
wrong_answer = "And if we keep going we'll see that all the numbers of the form 8n+2 are two-digit integers."

combined_responses = ""
for i in range(len(Answer)):
    combined_responses += Answer[i] + "\n"
    
combined_wrong_responses = combined_responses + wrong_answer

input_text = prompt.format(Question, Answer)
inputs = critic_tokenizer(combined_wrong_responses, return_tensors="pt").to("cuda")
with torch.no_grad():
    outputs = critic(**inputs)
print(outputs.logits)

tensor([[ 0.4651, -1.4561, -0.1776]], device='cuda:0')


In [8]:
#take the softmax of the logits
import torch.nn.functional as F
probabilities = F.softmax(outputs.logits, dim=1)
print(probabilities)


tensor([[0.5980, 0.0876, 0.3145]], device='cuda:0')


In [84]:
result = '### Input:\nWHow many seconds are in 7.8 minutes?\n\n### Response:\n'
responses = [ "7.8 minutes is the same as 7 minutes and 0.8 minutes.", "Right, and since there are 60 seconds in a minute, then there are 60 * 7 = 420 seconds in 7 minutes.", "And since there are 60 seconds in a minute, then there are 60 * 0.8 = 48 seconds in 0.8 minutes.", "So, in total, there are 420 + 48 = 468 seconds in 7.8 minutes." ]


In [31]:
result = ouput_text[0].split('### Response:\n')[0] + '### Response:\n'
responses = ouput_text[0].split('### Response:\n')[1].split('\n')

In [32]:
correct_probability = 1
incorrect_probability = 1
neutral_probability = 1

with torch.no_grad():
    for response in responses:
        result += response + "\n"
        inputs = critic_tokenizer(result, return_tensors="pt").to("cuda")
        outputs = critic(**inputs)
        #take the softmax of the logits
        import torch.nn.functional as F
        probabilities = F.softmax(outputs.logits, dim=1)
        print(probabilities)
        correct_probability *= (probabilities[0][2]+ probabilities[0][1])
        incorrect_probability *= probabilities[0][0]
        # neutral_probability *= 
    

    

tensor([[0.1941, 0.0384, 0.7675]], device='cuda:0')
tensor([[0.0710, 0.0134, 0.9156]], device='cuda:0')
tensor([[0.0393, 0.0118, 0.9489]], device='cuda:0')
tensor([[0.0174, 0.0075, 0.9751]], device='cuda:0')


In [33]:
print(correct_probability)
print(incorrect_probability)
# print(neutral_probability)

tensor(0.7068, device='cuda:0')
tensor(9.4226e-06, device='cuda:0')


In [34]:
responses

['First find how many bolts of blue fiber the robe takes: 2 bolts of blue fiber * 2/2 = <<2*2=4>>4 bolts',
 'Then find how many bolts of white fiber the robe takes: 4 bolts of white fiber / 2 = <<4/2=2>>2 bolts',
 'Then add the two bolt counts to find the total number of bolts: 2 bolts of blue fiber + 2 bolts of white fiber = <<2+2=4>>4 bolts',
 '#### 4']

In [44]:
inputs = critic_tokenizer(result, return_tensors="pt").to("cuda")
outputs = critic(**inputs)
#take the softmax of the logits
import torch.nn.functional as F
probabilities = F.softmax(outputs.logits, dim=1)
print(probabilities)


tensor([[0.4339, 0.2652, 0.3009]], device='cuda:0', grad_fn=<SoftmaxBackward0>)


In [41]:
probabilities_total = probabilities_total* probabilities
probabilities_total

tensor([[1.6243e-02, 6.9465e-06, 9.4159e-02]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [26]:
probabilities_total

tensor([[1.3050e-02, 1.7264e-06, 1.1865e-01]], device='cuda:0',
       grad_fn=<MulBackward0>)

In [13]:
output = ouput_text[0].split("<s> \n")[1].split("</s>")[0]

In [14]:
output

'### Input:\nA robe takes 2 bolts of blue fiber and half that much white fiber. How many bolts in total does it take?\n\n### Response:\nFirst find the total number of blue fiber: 2 bolts * 2 = <<2*2=4>>4 bolts\nThen find the total number of white fiber: 2 bolts / 2 = <<2/2=1>>1 bolt\nThen add the blue and white fiber to find the total number of fiber: 4 bolts + 1 bolt = <<4+1=5>>5 bolts\n#### 5'

In [15]:
#turn outputs into tokens
answer = critic_tokenizer(output, return_tensors="pt").to("cuda")

In [16]:
answer

{'input_ids': tensor([[    1,   835, 10567, 29901,    13, 29909,   696,   915,  4893, 29871,
         29906, 15772,  1372,   310,  7254,  5713,   495,   322,  4203,   393,
          1568,  4796,  5713,   495, 29889,  1128,  1784, 15772,  1372,   297,
          3001,   947,   372,  2125, 29973,    13,    13,  2277, 29937, 13291,
         29901,    13,  6730,  1284,   278,  3001,  1353,   310,  7254,  5713,
           495, 29901, 29871, 29906, 15772,  1372,   334, 29871, 29906,   353,
          3532, 29906, 29930, 29906, 29922, 29946,  6778, 29946, 15772,  1372,
            13, 11760,  1284,   278,  3001,  1353,   310,  4796,  5713,   495,
         29901, 29871, 29906, 15772,  1372,   847, 29871, 29906,   353,  3532,
         29906, 29914, 29906, 29922, 29896,  6778, 29896,   289, 14339,    13,
         11760,   788,   278,  7254,   322,  4796,  5713,   495,   304,  1284,
           278,  3001,  1353,   310,  5713,   495, 29901, 29871, 29946, 15772,
          1372,   718, 29871, 29896,  

In [18]:
outputs = critic(**answer)
print(outputs.logits)

AttributeError: 'LlamaModel' object has no attribute '_has_no_labels'